In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import pandas as pd

In [3]:
import GA

In [4]:
ls -arlt ../datasets/

total 12
drwxrwxr-x  3 lisa lisa 4096 Jul  8 01:23 ./
drwxrwxr-x  2 lisa lisa 4096 Jul  8 01:24 liuthyroid/
drwxrwxr-x 10 lisa lisa 4096 Jul 10 16:25 ../


In [5]:
yt=pd.read_csv("../datasets/liuthyroid/liu_thrb_orig.csv", index_col=0)
xt=pd.read_csv("../datasets/liuthyroid/liu_thrb_lb_dragon6.txt", sep="\t", index_col=0)

In [6]:
%%latex
$$Y_i = \beta_{i0} + \beta_{i 1} X_1 + \beta_{i2} X_2 + \ldots + \beta_{ip}X_p +\epsilon_i$$

<IPython.core.display.Latex object>

In [7]:
yt.head()

,GATS1e,EEig08x,EEig07d,GGI6,R6v+,H-051,Y Exp.,Y-Pred,Seta
ID,,,,,,,,,
1,0.722,2.372,1.846,0.771,0.042,0,9.59,9.84,1
2,0.710,1.869,1.846,0.567,0.046,2,10.32,10.43,1
3,0.709,2.372,2.145,0.771,0.031,0,9.96,10.62,1
4,0.705,1.984,1.846,0.690,0.049,2,10.72,10.47,1
5,0.732,2.334,2.145,0.690,0.033,2,10.60,9.56,1


In [8]:
yt=yt["Y Exp."].dropna()
xt=xt.dropna()
xt=xt.drop(columns=["NAME"])

In [9]:
xt.shape

(85, 1176)

In [10]:
yt.shape

(85,)

In [11]:
xt.head()

,MW,AMW,Sv,Mv,Me,Mp,Mi,nBM,RBN,RBF,...,DLS_07,DLS_cons,LLS_01,LLS_02,CMC-80,Inflammat-80,Psychotic-80,Hypertens-80,Neoplastic-80,Infective-80
No.,,,,,,,,,,,,,,,,,,,,,
1,650.98,18.599,26.519,0.758,1.023,0.889,1.097,13,5,0.139,...,1.0,0.65,0.17,0.88,0,0,0,0,0,0
3,567.18,12.890,29.782,0.677,1.009,0.767,1.112,13,6,0.133,...,1.0,0.88,0.17,0.88,0,0,0,0,0,0
4,635.96,19.272,25.497,0.773,1.021,0.913,1.088,13,5,0.147,...,1.0,0.61,0.17,0.75,0,0,0,0,0,0
5,458.16,10.909,28.178,0.671,1.014,0.718,1.111,13,6,0.140,...,1.0,0.87,0.17,0.88,0,0,0,0,0,0
6,430.10,11.947,25.124,0.698,1.023,0.739,1.107,13,4,0.108,...,1.0,0.87,0.17,1.00,1,0,0,0,0,0


In [12]:
xt.shape

(85, 1176)

In [13]:
yt.shape

(85,)

In [14]:
ga_inst1=GA.GAmlr.GAdescsel(xt,yt,ngen=10, popsize=100, indsize=5, cx=.5, mut=0.05, seed=int(12345))

In [15]:
pop_o, pop_f = ga_inst1.evolve(evalfunc="r2")

In [16]:
pop_f

,0,1,2,3,4,scores
0,H4s,RDF040m,J_G,B08[N-Br],B07[O-O],0.593101
1,SM14_AEA(bo),PW5,GATS4p,JGI4,nI,0.592793
2,HATS4s,ATSC6p,JGT,R4u+,SpMaxA_EA(dm),0.591657
3,GATS7e,SM11_AEA(bo),GATS4p,JGI4,nI,0.574063
4,GGI9,B10[C-Br],GATS4p,JGI4,SpMaxA_EA(dm),0.571478
5,SpMin2_Bh(e),SsssN,TDB05m,JGI4,R8v,0.566891
6,RDF055u,SM11_AEA(bo),GATS4p,JGI4,nI,0.555522
7,GGI9,RDF040m,GATS5e,SPI,R5m,0.554315
8,Mor12s,TDB03s,GATS4s,GGI6,SpMaxA_EA(dm),0.552353
9,SM14_AEA(bo),PW5,R4p,R3v,SpMaxA_EA(dm),0.551148


In [17]:
ga_inst2=GA.GAmlr.GAdescsel(xt,yt,ngen=1000, popsize=100, indsize=5, cx=.5, mut=0.05, seed=int(12345))

In [18]:
pop_o2, pop_f2 = ga_inst2.evolve(evalfunc="r2")